In [1]:
import os
import zipfile
import csv

import requests

def get_data():
    return csv.DictReader((x for x in open("data/ventas_2016_atributos.csv",encoding = "utf-8-sig")), delimiter=";",)

def get_ratings():
    return get_data()

def get_book_features():
    return csv.DictReader((x for x in open("data/ventas_2016_atributos.csv",encoding = "utf-8-sig")), delimiter=";",)

In [2]:
import json
from itertools import islice

ratings = get_data()

In [3]:
for line in islice(ratings, 2):
    print(json.dumps(line, indent=4))

{
    "A\u00d1O": "2016",
    "NIT": "70351028",
    "INACOD": "5",
    "SIGLA_COMERCIAL": "AUTOSERVICIO MERCARAPIDO                ",
    "CiuNom": "LA DORADA",
    "canal": "SUPERETE",
    "subcanal": "NO TIENE SUB CANAL",
    "tama\u00f1o": "A",
    "Punto": "NULL",
    "barrio": "CENTRO                                  ",
    "COD_VENDED": "B14",
    "CODIGO_ARTICULO": "101021",
    "ARTICULO": "ACEITUNA RELLENA PIMENTON CORU\u00d1A",
    "PreArtNom": "24 FCO X 250 GR                                             ",
    "COD_GRUPO": "101",
    "GRUPO": "INDUSTRIAS LA CORU\u00d1A",
    "CODIGO_SUBGRUPO": "101-001        ",
    "SUBGRUPO": "CONSERVAS                   ",
    "CODIGO_FAMILIA": "101-001-0100   ",
    "UND_NEGOCIO": "TRAD. POBLACIONES 1 B/MANGA",
    "FAMILIA": "ACEITUNAS                               ",
    "EMBALAJE": "24",
    "NUM_VECES_COMP": "1"
}
{
    "A\u00d1O": "2016",
    "NIT": "88272013",
    "INACOD": "12",
    "SIGLA_COMERCIAL": "AUTOSERVICIO MERCANTIL AHOR

In [4]:
from lightfm.data import Dataset

dataset = Dataset()
dataset.fit((x['NIT'] for x in get_ratings()),
            (x['CODIGO_ARTICULO'] for x in get_ratings()))

In [5]:
num_users, num_items = dataset.interactions_shape()
print('Num users: {}, num_items {}.'.format(num_users, num_items))

Num users: 12474, num_items 3207.


In [6]:
dataset.fit_partial(items=(x['CODIGO_ARTICULO'] for x in get_ratings()),
                    item_features=((x['FAMILIA'],x['EMBALAJE']) for x in get_book_features()))


In [ ]:
(interactions, weights) = dataset.build_interactions(((x['NIT'], x['CODIGO_ARTICULO'],float(x['NUM_VECES_COMP']))
                                                      for x in get_ratings()))

print(repr(interactions))

In [ ]:
print(weights)

In [ ]:
item_features = dataset.build_item_features(((x['CODIGO_ARTICULO'], [(x['FAMILIA'], x['EMBALAJE'])]) 
                                             for x in get_book_features()))
print(repr(item_features))

In [ ]:
print(item_features)

In [ ]:
from lightfm import LightFM

model = LightFM(loss='bpr')
model.fit(weights, item_features=item_features)

In [ ]:
import numpy as np

n_users,n_items = interactions.shape
print('shape iteractions: ',interactions.todense().shape)

eval_user = 1

scores = model.predict(eval_user, np.arange(n_items))
print('scores: ',scores.shape)
print('negative scores: ',-scores)
print('position of items: ',np.argsort(-scores))

top_items = interactions.todense()[eval_user,np.argsort(-scores)]
print('sorted items: ',top_items)